In [155]:
import numpy as np
import pandas as pd 
import scipy.stats as ss
import random
import pyarrow as pa

from pyarrow import csv

In [88]:
def pad_data(df, cols):
    df[cols[np.invert(np.isin(cols, df.columns))]] = 0
    return df

def merge(in_file, filename, cols, skip_first=True):
    chunksize = 10 ** 6
    with csv.open_csv(
        in_file,
        convert_options = csv.ConvertOptions(),
        read_options=csv.ReadOptions(
            use_threads=True,
            block_size=chunksize
        )) as reader:
        

        for chunk in reader:
            chunk = chunk.to_pandas()
            if skip_first:
                chunk = chunk[chunk.columns[1:]]
            tmp = pd.DataFrame(columns=cols)
            chunk = append_dataset(tmp, chunk)
            chunk.to_csv(filename, mode='a', header=False)

In [89]:
def default_transform(df):
    return df

def read_chunk(name, chunk_id, opts=csv.ConvertOptions(), transform=default_transform, extra=1):
    chunk=None
    chunksize = extra*10 ** 7
    with csv.open_csv(
        name,
        convert_options = opts,
        read_options=csv.ReadOptions(
            use_threads=True,
            block_size=chunksize
        )) as reader:

        i=0
        for next_chunk in reader:
            if next_chunk is None:
                break
            chunk=next_chunk
            if i == chunk_id:
                return transform(chunk.to_pandas())
            i+=1
            
    return None

In [164]:
cols = read_chunk('data/split/NF-UNSW-NB15-V2-TRAIN.csv', 0).columns.drop([''])

In [165]:
cols = np.unique(np.concatenate((cols, read_chunk('data/split/ddos2019-TRAIN.csv', 0).columns.drop(['']))))

In [170]:
cols

array([' ACK Flag Count', ' Active Max', ' Active Min', ' Active Std',
       ' Average Packet Size', ' Avg Bwd Segment Size',
       ' Avg Fwd Segment Size', ' Bwd Avg Bytes/Bulk',
       ' Bwd Avg Packets/Bulk', ' Bwd Header Length', ' Bwd IAT Max',
       ' Bwd IAT Mean', ' Bwd IAT Min', ' Bwd IAT Std', ' Bwd PSH Flags',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Min',
       ' Bwd Packet Length Std', ' Bwd Packets/s', ' Bwd URG Flags',
       ' CWE Flag Count', ' Down/Up Ratio', ' ECE Flag Count',
       ' Flow IAT Max', ' Flow IAT Mean', ' Flow IAT Min',
       ' Flow IAT Std', ' Flow Packets/s', ' Fwd Avg Bulk Rate',
       ' Fwd Avg Packets/Bulk', ' Fwd Header Length',
       ' Fwd Header Length.1', ' Fwd IAT Max', ' Fwd IAT Mean',
       ' Fwd IAT Min', ' Fwd IAT Std', ' Fwd Packet Length Max',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Min',
       ' Fwd Packet Length Std', ' Fwd URG Flags', ' Idle Max',
       ' Idle Min', ' Idle Std', ' Inbound', ' Init

In [175]:
pd.DataFrame(columns=cols).to_csv('data/split/MERGED_DDOS_NFUQ-TRAIN.csv')

In [ ]:
merge('data/split/NF-UQ-NIDS-v2-TRAIN.csv', 'data/split/MERGED_DDOS_NFUQ-TRAIN.csv', cols)

/tmp/ipykernel_287990/1439384176.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return dataset.append(df, ignore_index=True)
/tmp/ipykernel_287990/1439384176.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return dataset.append(df, ignore_index=True)
/tmp/ipykernel_287990/1439384176.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return dataset.append(df, ignore_index=True)
/tmp/ipykernel_287990/1439384176.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return dataset.append(df, ignore_index=True)
/tmp/ipykernel_287990/1439384176.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas 

In [ ]:
merge('data/split/ddos2019-TRAIN.csv', 'data/split/MERGED_DDOS_NFUQ-TRAIN.csv', cols)

In [9]:
cols

array([' ACK Flag Count', ' Active Max', ' Active Min', ' Active Std',
       ' Average Packet Size', ' Avg Bwd Segment Size',
       ' Avg Fwd Segment Size', ' Bwd Avg Bytes/Bulk',
       ' Bwd Avg Packets/Bulk', ' Bwd Header Length', ' Bwd IAT Max',
       ' Bwd IAT Mean', ' Bwd IAT Min', ' Bwd IAT Std', ' Bwd PSH Flags',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Min',
       ' Bwd Packet Length Std', ' Bwd Packets/s', ' Bwd URG Flags',
       ' CWE Flag Count', ' Down/Up Ratio', ' ECE Flag Count',
       ' Flow IAT Max', ' Flow IAT Mean', ' Flow IAT Min',
       ' Flow IAT Std', ' Flow Packets/s', ' Fwd Avg Bulk Rate',
       ' Fwd Avg Packets/Bulk', ' Fwd Header Length',
       ' Fwd Header Length.1', ' Fwd IAT Max', ' Fwd IAT Mean',
       ' Fwd IAT Min', ' Fwd IAT Std', ' Fwd Packet Length Max',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Min',
       ' Fwd Packet Length Std', ' Fwd URG Flags', ' Idle Max',
       ' Idle Min', ' Idle Std', ' Inbound', ' Init

In [43]:
df[df.columns.drop('')]

,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,Bwd Packet Length Max,FIN Flag Count,Flow Bytes/s,Fwd Avg Bytes/Bulk,Fwd PSH Flags,Fwd Packets/s,Idle Mean,Init_Win_bytes_forward,Subflow Fwd Packets,mask
0,1305,21,6,1.0,9,1,193,3,24,24,...,0,0,0,0,0,0,0,0,0,0
1,1305,21,6,1.0,261,5,469,7,24,24,...,0,0,0,0,0,0,0,0,0,0
2,1305,21,6,1.0,481,9,750,11,24,24,...,0,0,0,0,0,0,0,0,0,0
3,1305,21,6,1.0,701,13,1054,15,24,24,...,0,0,0,0,0,0,0,0,0,0
4,1305,21,6,1.0,1031,19,1474,21,24,24,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61056,56298,21,6,1.0,481,9,750,11,24,24,...,0,0,0,0,0,0,0,0,0,0
61057,14236,21,6,1.0,2059,37,2816,39,24,24,...,0,0,0,0,0,0,0,0,0,0
61058,56298,21,6,1.0,701,13,1054,15,24,24,...,0,0,0,0,0,0,0,0,0,0
61059,14236,21,6,1.0,2383,43,3234,45,24,24,...,0,0,0,0,0,0,0,0,0,0


In [13]:
pad_data(df, cols)['Attack']

0        Benign
1        Benign
2        Benign
3        Benign
4        Benign
          ...  
61056    Benign
61057    Benign
61058    Benign
61059    Benign
61060    Benign
Name: Attack, Length: 61061, dtype: object

In [58]:
dfu = read_chunk('data/split/NF-UNSW-NB15-V2-TRAIN.csv', 0)
dfu[dfu.columns.drop('')].to_csv('data/split/MERGED_DDOS_UNSW-TRAIN.csv', mode='a', header=False)

In [105]:
df = pd.DataFrame(columns=cols)

In [60]:
dfu= dfu[dfu.columns.drop('')]

In [67]:
df

,ACK Flag Count,Active Max,Active Min,Active Std,Average Packet Size,Avg Bwd Segment Size,Avg Fwd Segment Size,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Header Length,...,RETRANSMITTED_OUT_PKTS,SERVER_TCP_FLAGS,SHORTEST_FLOW_PKT,SRC_TO_DST_AVG_THROUGHPUT,SRC_TO_DST_SECOND_BYTES,Subflow Fwd Packets,TCP_FLAGS,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,mask


In [106]:
def append_dataset(dataset, df):
    df[cols[np.invert(np.isin(cols, df.columns))]] = 0
    return dataset.append(df, ignore_index=True)

In [61]:
dfu

,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label,Attack
0,1305,21,6,1.0,9,1,193,3,24,24,...,0,7240,0,0,0,0,0,331.0,0,Benign
1,1305,21,6,1.0,261,5,469,7,24,24,...,8688,8688,18944,74,0,0,0,230.0,0,Benign
2,1305,21,6,1.0,481,9,750,11,24,24,...,10136,10136,33792,132,0,0,0,229.0,0,Benign
3,1305,21,6,1.0,701,13,1054,15,24,24,...,11584,11584,48640,190,0,0,0,125.0,0,Benign
4,1305,21,6,1.0,1031,19,1474,21,24,24,...,14480,13032,64256,251,0,0,0,230.0,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61056,56298,21,6,1.0,481,9,750,11,24,24,...,10136,10136,33792,132,0,0,0,229.0,0,Benign
61057,14236,21,6,1.0,2059,37,2816,39,24,24,...,21720,18824,63744,249,0,0,0,125.0,0,Benign
61058,56298,21,6,1.0,701,13,1054,15,24,24,...,11584,11584,48640,190,0,0,0,125.0,0,Benign
61059,14236,21,6,1.0,2383,43,3234,45,24,24,...,24616,20272,13056,51,0,0,0,221.0,0,Benign


In [168]:
dfr = read_chunk('data/split/NF-UQ-NIDS-v2-TRAIN.csv', 0)

In [85]:
append_dataset(df, dfu).to_csv('data/split/MERGED_DDOS_UNSW-TRAIN.csv', mode='a', header=False)

/tmp/ipykernel_287990/1439384176.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return dataset.append(df, ignore_index=True)


In [169]:
dfr

,,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,...,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label,Dataset,mask
0,0,192.168.100.148,192.168.100.7,6,7.0,420,3,0,0,2,...,0,35840,140,0,0,0,0.0,1,NF-BoT-IoT-v2,1
1,1,192.168.100.148,192.168.100.5,6,7.0,280,2,40,1,22,...,0,0,0,0,0,0,0.0,1,NF-BoT-IoT-v2,1
2,2,192.168.1.31,192.168.1.79,6,0.0,44,1,40,1,22,...,0,0,0,0,0,0,0.0,0,NF-ToN-IoT-v2,1
3,3,192.168.1.34,192.168.1.79,6,0.0,44,1,40,1,22,...,0,0,0,0,0,0,0.0,0,NF-ToN-IoT-v2,1
4,4,192.168.1.30,192.168.1.152,6,0.0,44,1,40,1,22,...,0,0,0,0,0,0,0.0,0,NF-ToN-IoT-v2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58457,3144,59.166.0.4,149.171.126.5,6,7.0,19514,366,1087890,746,27,...,7240,8448,33,0,0,0,0.0,0,NF-UNSW-NB15-v2,1
58458,3145,59.166.0.2,149.171.126.9,17,0.0,568,4,312,4,0,...,0,0,0,0,0,0,0.0,0,NF-UNSW-NB15-v2,1
58459,3146,192.168.100.148,192.168.100.3,17,188.0,140,5,0,0,0,...,0,0,0,0,0,0,0.0,1,NF-BoT-IoT-v2,1
58460,3147,192.168.100.149,192.168.100.6,17,188.0,56,2,0,0,0,...,0,0,0,0,0,0,0.0,1,NF-BoT-IoT-v2,1


In [162]:
dft = read_chunk('data/split/MERGED_DDOS_UNSW-TRAIN.csv', 1100, csv.ConvertOptions(column_types=construct_read_dict(cols)))

In [ ]:
t = append_dataset(tmp, dfr[dfr.columns.drop([''])])

In [158]:
def construct_read_dict(cols):
    d = dict()
    for col in cols:
        if not col in ['Label', 'Attack', 'mask']:
            d[col] = pa.float32()
    return d

In [159]:
construct_read_dict(cols)

{' ACK Flag Count': DataType(float),
 ' Active Max': DataType(float),
 ' Active Min': DataType(float),
 ' Active Std': DataType(float),
 ' Average Packet Size': DataType(float),
 ' Avg Bwd Segment Size': DataType(float),
 ' Avg Fwd Segment Size': DataType(float),
 ' Bwd Avg Bytes/Bulk': DataType(float),
 ' Bwd Avg Packets/Bulk': DataType(float),
 ' Bwd Header Length': DataType(float),
 ' Bwd IAT Max': DataType(float),
 ' Bwd IAT Mean': DataType(float),
 ' Bwd IAT Min': DataType(float),
 ' Bwd IAT Std': DataType(float),
 ' Bwd PSH Flags': DataType(float),
 ' Bwd Packet Length Mean': DataType(float),
 ' Bwd Packet Length Min': DataType(float),
 ' Bwd Packet Length Std': DataType(float),
 ' Bwd Packets/s': DataType(float),
 ' Bwd URG Flags': DataType(float),
 ' CWE Flag Count': DataType(float),
 ' Down/Up Ratio': DataType(float),
 ' ECE Flag Count': DataType(float),
 ' Flow IAT Max': DataType(float),
 ' Flow IAT Mean': DataType(float),
 ' Flow IAT Min': DataType(float),
 ' Flow IAT Std': 

In [163]:
t

In [120]:
t.to_csv('data/split/MERGED_DDOS_UNSW-TRAIN.csv', mode='a', header=False)

In [116]:
tmp = pd.DataFrame(columns=cols)

In [110]:
t = append_dataset(tmp, dfr)

/tmp/ipykernel_287990/1439384176.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return dataset.append(df, ignore_index=True)


In [112]:
t.columns.drop([''])

Index([' ACK Flag Count', ' Active Max', ' Active Min', ' Active Std',
       ' Average Packet Size', ' Avg Bwd Segment Size',
       ' Avg Fwd Segment Size', ' Bwd Avg Bytes/Bulk', ' Bwd Avg Packets/Bulk',
       ' Bwd Header Length',
       ...
       'SERVER_TCP_FLAGS', 'SHORTEST_FLOW_PKT', 'SRC_TO_DST_AVG_THROUGHPUT',
       'SRC_TO_DST_SECOND_BYTES', 'Subflow Fwd Packets', 'TCP_FLAGS',
       'TCP_WIN_MAX_IN', 'TCP_WIN_MAX_OUT', 'mask', ''],
      dtype='object', length=116)

In [113]:
dfr.columns

Index(['', 'PROTOCOL', 'FLOW_DURATION_MILLISECONDS', 'IN_PKTS', 'OUT_PKTS',
       'IN_BYTES', 'OUT_BYTES', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ...
       'RETRANSMITTED_IN_PKTS', 'RETRANSMITTED_OUT_BYTES',
       'RETRANSMITTED_OUT_PKTS', 'SERVER_TCP_FLAGS', 'SHORTEST_FLOW_PKT',
       'SRC_TO_DST_AVG_THROUGHPUT', 'SRC_TO_DST_SECOND_BYTES', 'TCP_FLAGS',
       'TCP_WIN_MAX_IN', 'TCP_WIN_MAX_OUT'],
      dtype='object', length=116)

In [143]:
dft.columns[70]

'DST_TO_SRC_SECOND_BYTES'